In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from projection_stg import *
from utils import *
from sklearn import linear_model
import seaborn as sns
import pandas as pd
from pycasso.core import Solver as ScadSolver

device = "cpu"

In [ ]:
n = 40
d_vec = np.arange(30) + 2
n_simulations = 100
std_n = 1.0

print(d_vec)

In [ ]:
error_mat = np.zeros((n_simulations, len(d_vec)))
acc_mat_stg = np.zeros((n_simulations, len(d_vec)))
for j, d in enumerate(d_vec):
    for i in range(n_simulations):

        # number of features of X
        D = 64

        # dimension of y
        d2 = 1

        X = torch.randn((n, D), dtype=torch.float)

        theta = 2 * torch.randint(low=0, high=2, size=(D, d2), dtype=torch.float) - 1

        theta_new = theta
        theta_new[d:] = 0

        y = torch.mm(X, theta_new)
        noise = torch.randn((n, d2), dtype=torch.float) * std_n
        y += noise

        lam = 0.2 * np.sqrt((std_n ** 2) * np.log(D) * np.log(D - d) / n)

        theta_hat_mu = stg_proj(
            X, y, lam=lam, n_iter=4000, n_est=50, learning_rate=1e-3
        )

        theta_hat_mu = theta_hat_mu.detach().numpy()[0].reshape(-1)
        theta_new = theta_new.detach().numpy().reshape(-1)
        indices = np.argsort(-np.abs(theta_hat_mu))[d:]
        support_stg_proj = np.copy(theta_hat_mu)
        support_stg_proj[indices] = 0

        error_mat[i, j] = sum(
            (theta_hat_mu.reshape(-1) - theta_new.reshape(-1)) ** 2
        ) / sum((theta_new) ** 2)
        acc_mat_stg[i, j] = (
            np.mean(
                np.sign(np.abs(support_stg_proj.reshape(-1)))
                == np.sign(np.abs(theta_new.reshape(-1)))
            )
            == 1
        )

# LASSO

In [ ]:
my_acc_mat = np.zeros((n_simulations, len(d_vec)))
acc_mat_lasso = np.zeros((n_simulations, len(d_vec)))
lasso_error_mat = np.zeros((n_simulations, len(d_vec)))
for j, d in enumerate(d_vec):
    my_acc_avg = 0
    lasso_acc_avg = 0
    per_rec = 0
    per_rec_my = 0
    per_rec_my_stg = 0
    for i in range(0, n_simulations):
        D = 64

        x_0 = np.zeros((n, D))
        sig = np.sign(np.random.normal(size=(1, d)))
        A = np.random.normal(size=(n, D))

        x_r = np.hstack((sig, np.zeros((1, D - d))))
        x_r = x_r[0]
        x_r = np.random.permutation(x_r)

        x_r = x_r.reshape(-1, 1)

        y = np.matmul(A, x_r) + (std_n ** 1) * np.random.normal(size=(n, 1))
        lambda_1 = np.sqrt(1 * (std_n ** 2) * np.log(d) * np.log(D - d) / n)
        lass = linear_model.Lasso(alpha=lambda_1)
        lass.fit(A, y)

        linear_model.Lasso(
            alpha=lambda_1,
            copy_X=True,
            fit_intercept=True,
            max_iter=200,
            normalize=False,
            positive=False,
            precompute=False,
            random_state=None,
            selection="cyclic",
            tol=0.0001,
            warm_start=False,
        )
        indices = np.argsort(-np.abs(lass.coef_))[d:]
        support_lass = np.copy(lass.coef_)
        support_lass[indices] = 0
        lasso_error_mat[i, j] = sum(
            (lass.coef_.reshape(-1) - x_r.reshape(-1)) ** 2
        ) / sum((x_r ** 2))
        acc_mat_lasso[i, j] = (
            np.mean(np.sign(np.abs(support_lass)) == np.sign(np.abs(x_r.T))) == 1
        )

# OMP

In [ ]:
acc_mat_omp = np.zeros((n_simulations, len(d_vec)))
omp_error_mat = np.zeros((n_simulations, len(d_vec)))
const = 1
for j, d in enumerate(d_vec):
    my_acc_avg = 0
    lasso_acc_avg = 0
    per_rec = 0
    per_rec_my = 0
    per_rec_my_stg = 0
    for i in range(0, n_simulations):
        D = 64

        x_0 = np.zeros((n, D))
        sig = 1 * np.sign(np.random.normal(size=(1, d)))  #
        A = np.random.normal(size=(n, D))
        x_r = np.hstack((sig, np.zeros((1, D - d))))
        x_r = x_r[0]
        x_r = x_r.reshape(-1, 1)

        # var=0.5
        y = np.matmul(A, x_r) + (std_n ** 1) * np.random.normal(size=(n, 1))
        omp_coef = omp_orig(A, y, const, nonneg=False, ncoef=d)

        coef = omp_coef.coef

        acc_mat_omp[i, j] = (
            np.mean(np.sign(np.abs(coef)) == np.sign(np.abs(x_r.T))) == 1
        )
        omp_error_mat[i, j] = sum((coef.reshape(-1) - x_r.reshape(-1)) ** 2) / sum(
            (x_r ** 2)
        )

# Random OMP

In [ ]:
acc_mat_rand_omp = np.zeros((n_simulations, len(d_vec)))
rand_omp_error_mat = np.zeros((n_simulations, len(d_vec)))
const = 4 / 3
N_runs = 500
for j, d in enumerate(d_vec):
    my_acc_avg = 0
    lasso_acc_avg = 0
    per_rec = 0
    per_rec_my = 0
    per_rec_my_stg = 0
    for i in range(0, n_simulations):
        D = 64

        x_0 = np.zeros((n, D))
        sig = 1 * np.sign(np.random.normal(size=(1, d)))  # np.ones((1,d))#
        A = np.random.normal(size=(n, D))
        x_r = np.hstack((sig, np.zeros((1, D - d))))
        x_r = x_r[0]
        x_r = x_r.reshape(-1, 1)

        # var=0.5
        y = np.matmul(A, x_r) + (std_n ** 1) * np.random.normal(size=(n, 1))
        omp_coef = np.zeros((D,))
        for l in range(N_runs):
            omp_res = omp(A, y, const, nonneg=False, ncoef=d)
            omp_coef += omp_res.coef
        omp_coef = omp_coef / N_runs
        indices = np.argsort(-np.abs(omp_coef))[d:]
        support_romp = np.copy(omp_coef)
        support_romp[indices] = 0

        acc_mat_rand_omp[i, j] = (
            np.mean(np.sign(np.abs(support_romp)) == np.sign(np.abs(x_r.T))) == 1
        )
        rand_omp_error_mat[i, j] = sum(
            (omp_coef.reshape(-1) - x_r.reshape(-1)) ** 2
        ) / sum((x_r ** 2))
        # print(j)

# SCAD

In [ ]:
acc_mat_scad = np.zeros((n_simulations, len(d_vec)))
scad_error_mat = np.zeros((n_simulations, len(d_vec)))
const = 4 / 3
N_runs = 500
for j, d in enumerate(d_vec):
    my_acc_avg = 0
    lasso_acc_avg = 0
    per_rec = 0
    per_rec_my = 0
    per_rec_my_stg = 0
    for i in range(0, n_simulations):
        D = 64

        x_0 = np.zeros((n, D))
        sig = 1 * np.sign(np.random.normal(size=(1, d)))  # np.ones((1,d))#
        A = np.random.normal(size=(n, D))
        x_r = np.hstack((sig, np.zeros((1, D - d))))
        x_r = x_r[0]
        x_r = x_r.reshape(-1, 1)

        y = np.matmul(A, x_r) + (std_n ** 1) * np.random.normal(size=(n, 1))

        scad_solver = ScadSolver(A, y, penalty="scad")
        scad_solver.train()

        scad_dict = scad_solver.coef()

        if np.where(scad_dict["df"] == d)[0].__len__() > 0:
            best_ind = np.where(scad_dict["df"] == d)[0][
                np.argmin(
                    np.sum(
                        (A @ scad_dict["beta"][np.where(scad_dict["df"] == d)].T - y)
                        ** 2,
                        axis=0,
                    )
                )
            ]
            scad_support = np.sign(
                np.abs((scad_dict["beta"][best_ind, :]))
            )  # np.sign(np.abs(np.mean(scad_dict['beta'][np.where(scad_dict['df'] == d)], axis=0)))
            acc_mat_scad[i, j] = np.mean(scad_support == np.sign(np.abs(x_r.T))) == 1
            scad_error_mat[i, j] = sum(
                (scad_dict["beta"][best_ind, :].reshape(-1) - x_r.reshape(-1)) ** 2
            ) / sum((x_r ** 2))
        else:
            acc_mat_scad[i, j] = 0
            scad_error_mat[i, j] = 1

In [ ]:
sns.reset_orig()

sparsity = np.tile(np.arange(len(d_vec)) + 1, n_simulations).reshape(-1, 1)

dfs = []

tmp = [
    sparsity,
    acc_mat_stg.reshape(-1, 1),
    np.array(["Proj-STG"] * len(sparsity)).reshape(-1, 1),
]
tmp = np.concatenate(tmp, axis=1)
dfs.append(pd.DataFrame(tmp, columns=["Sparsity", "Success Rate", "Method"]))

tmp = [
    sparsity,
    acc_mat_lasso.reshape(-1, 1),
    np.array(["LASSO"] * len(sparsity)).reshape(-1, 1),
]
tmp = np.concatenate(tmp, axis=1)
dfs.append(pd.DataFrame(tmp, columns=["Sparsity", "Success Rate", "Method"]))

tmp = [
    sparsity,
    acc_mat_omp.reshape(-1, 1),
    np.array(["OMP"] * len(sparsity)).reshape(-1, 1),
]
tmp = np.concatenate(tmp, axis=1)
dfs.append(pd.DataFrame(tmp, columns=["Sparsity", "Success Rate", "Method"]))

tmp = [
    sparsity,
    acc_mat_rand_omp.reshape(-1, 1),
    np.array(["Rand-OMP"] * len(sparsity)).reshape(-1, 1),
]
tmp = np.concatenate(tmp, axis=1)
dfs.append(pd.DataFrame(tmp, columns=["Sparsity", "Success Rate", "Method"]))

tmp = [
    sparsity,
    acc_mat_scad.reshape(-1, 1),
    np.array(["SCAD"] * len(sparsity)).reshape(-1, 1),
]
tmp = np.concatenate(tmp, axis=1)
dfs.append(pd.DataFrame(tmp, columns=["Sparsity", "Success Rate", "Method"]))

df = pd.concat(dfs)
df = df.reset_index()
df = df.convert_dtypes()
df["Sparsity"] = df["Sparsity"].apply(lambda x: int(x))
df["Success Rate"] = df["Success Rate"].apply(lambda x: float(x))

sns.set(rc={"figure.figsize": (24, 10)})
sns.set_style(style="white")
p = sns.lineplot(
    data=df,
    x="Sparsity",
    y="Success Rate",
    markers=True,
    size="Method",
    sizes=(2, 3),
    hue="Method",
    style="Method",
    palette="viridis",
    ci=90,
)
p.set_title("$\sigma = 1.0$", fontsize=60)
p.set_xlabel("Sparsity", fontsize=60)
p.set_ylabel("Success Rate", fontsize=60)
p.set(xlim=(2, 25))
p.set_xticklabels(["{:n}".format(x) for x in p.get_xticks()], size=30)
p.set_yticklabels(["{:.1f}".format(x) for x in plt.yticks()[0]], size=30)
plt.legend(labels=["Proj-STG", "LASSO", "OMP", "Rand-OMP", "SCAD"], fontsize=40)